# Changing Music Preferences
### Do Spotify users listen to different music during the COVID-19 pandemic? 
### Does this relate to the amount of confirmed cases?

To Do: 

1. Scraping Spotify Top 200 data
2. Get song metadata through Spotify API
3. Data Visualization & Modelling
4. .....
5. Create interactive dashboard in Shiny


## 1.
## Scraping the Top 200 data for 2019 (baseline) and 2020 (pandemic).

https://spotifycharts.com contains the official Spotify Charts. 

Luckily, each day/week is available as a .csv file!
<br> <br> <br>
**Goal: get all the weekly .csv files from 2019 and 2020 and combine them to one.**

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io

In [ ]:
url = 'https://spotifycharts.com/regional/nl/weekly/latest'

r = requests.get(url)

soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
dates = [item["data-value"] for item in soup.find_all(attrs={"data-value": True}) if item["data-value"].startswith('2')]
dates[:5]

In [ ]:
urls = ["https://spotifycharts.com/regional/nl/weekly/" + date + "/download" for date in dates]
urls[:5]

### Warning: running the block below will result in scraping 200 .csv's!

In [ ]:
data = []

for date in tqdm(dates): 
    url = "https://spotifycharts.com/regional/nl/weekly/" + date + "/download"
    response = requests.get(url)
    file_object = io.StringIO(response.content.decode('utf-8'))
    df = pd.read_csv(file_object, header=1)
    df["Date"] = date
    data.append(df)

df = pd.concat(data)
df.reset_index(drop=True, inplace=True)
df.to_csv("..\\data\\raw\\top200_2017_2020.csv", index=False)
data = []

In [108]:
df = pd.read_csv("..\\data\\raw\\top200_2017_2020.csv")
df

,Position,Track Name,Artist,Streams,URL,Date
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30
2,3,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,1429665,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,2020-10-23--2020-10-30
3,4,Lonely (with benny blanco),Justin Bieber,1330793,https://open.spotify.com/track/4y4spB9m0Q6026K...,2020-10-23--2020-10-30
4,5,Head & Heart (feat. MNEK),Joel Corry,1271413,https://open.spotify.com/track/6cx06DFPPHchuUA...,2020-10-23--2020-10-30
...,...,...,...,...,...,...
40195,196,Sex,Cheat Codes,114030,https://open.spotify.com/track/5DA77EqppDmCTWG...,2016-12-23--2016-12-30
40196,197,Ain't My Fault,Zara Larsson,113974,https://open.spotify.com/track/0ADG9OgdVTL7fgR...,2016-12-23--2016-12-30
40197,198,Please Come Home for Christmas,Luther Vandross,113779,https://open.spotify.com/track/2mOtx6P21hecOcP...,2016-12-23--2016-12-30
40198,199,Jodge Me Niet - Titelsong Van De Film “SOOF 2”,Jayh,113763,https://open.spotify.com/track/2VxAfqI3vIOaPSl...,2016-12-23--2016-12-30


In [109]:
print(df.isna().sum())
df.dropna(inplace=True) # removing 8 rows out of 40k is justified

df[['Start Week', 'End Week']] = df['Date'].str.split('--', 1, expand=True) # add columns for start and end dates
df['ID'] = df['URL'].str.rsplit('https://open.spotify.com/track/', 1, expand=True)[1] # add ID column for later merge

df.to_csv("..\\data\\processed\\top200_2017_2020.csv", index=False)

Position      0
Track Name    7
Artist        7
Streams       0
URL           2
Date          0
dtype: int64


In [80]:
df = pd.read_csv("..\\data\\processed\\top200_2017_2020.csv")
df.head()

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4eWeOwo0TB0vkdrUhy4zbM
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,3tjFYV6RSFtuktYl3ZtYcq
2,3,"Lemonade (feat. Gunna, Don Toliver & NAV)",Internet Money,1429665,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,7hxHWCCAIIxFLCzvDgnQHX
3,4,Lonely (with benny blanco),Justin Bieber,1330793,https://open.spotify.com/track/4y4spB9m0Q6026K...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4y4spB9m0Q6026KfkAvy9Q
4,5,Head & Heart (feat. MNEK),Joel Corry,1271413,https://open.spotify.com/track/6cx06DFPPHchuUA...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,6cx06DFPPHchuUAcTxznu9


## 2.
## Retrieve and Combine music features through Spotify API


In [110]:
# Fill this in using your Spotify API Credentials from https://developer.spotify.com/dashboard/applications
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="XXX" 
secret = "XXX"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [112]:
# Where I save my Spotify API Credentials, outside of this public repository :)
import spotipy
%run credentials.py

In [113]:
# from https://github.com/tgel0/spotify-data/blob/master/notebooks/SpotifyDataRetrieval.ipynb under MIT Licence.

rows = []
batchsize = 100
None_counter = 0

for i in tqdm(range(0,len(df['ID']),batchsize)):
    batch = df['ID'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

100%|██████████| 402/402 [00:39<00:00, 10.19it/s]


In [114]:
features = pd.DataFrame.from_dict(rows, orient="columns")
features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.781,0.412,0,-10.860,1,0.1410,0.26200,0.000000,0.4280,0.8790,159.981,audio_features,4eWeOwo0TB0vkdrUhy4zbM,spotify:track:4eWeOwo0TB0vkdrUhy4zbM,https://api.spotify.com/v1/tracks/4eWeOwo0TB0v...,https://api.spotify.com/v1/audio-analysis/4eWe...,278000,4
1,0.700,0.722,7,-3.558,0,0.0369,0.22100,0.000000,0.2720,0.7560,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4
2,0.800,0.658,1,-6.142,0,0.0790,0.25000,0.000000,0.1110,0.4620,140.042,audio_features,7hxHWCCAIIxFLCzvDgnQHX,spotify:track:7hxHWCCAIIxFLCzvDgnQHX,https://api.spotify.com/v1/tracks/7hxHWCCAIIxF...,https://api.spotify.com/v1/audio-analysis/7hxH...,195429,4
3,0.631,0.239,11,-7.071,0,0.0398,0.86400,0.000000,0.1160,0.0927,79.859,audio_features,4y4spB9m0Q6026KfkAvy9Q,spotify:track:4y4spB9m0Q6026KfkAvy9Q,https://api.spotify.com/v1/tracks/4y4spB9m0Q60...,https://api.spotify.com/v1/audio-analysis/4y4s...,149297,4
4,0.734,0.874,8,-3.158,1,0.0662,0.16800,0.000011,0.0489,0.9050,122.953,audio_features,6cx06DFPPHchuUAcTxznu9,spotify:track:6cx06DFPPHchuUAcTxznu9,https://api.spotify.com/v1/tracks/6cx06DFPPHch...,https://api.spotify.com/v1/audio-analysis/6cx0...,166028,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40188,0.510,0.692,0,-5.825,1,0.1710,0.00451,0.000000,0.1380,0.2090,102.420,audio_features,5DA77EqppDmCTWGovd4oTP,spotify:track:5DA77EqppDmCTWGovd4oTP,https://api.spotify.com/v1/tracks/5DA77EqppDmC...,https://api.spotify.com/v1/audio-analysis/5DA7...,228361,4
40189,0.576,0.782,6,-4.825,0,0.0296,0.00778,0.000000,0.2850,0.3550,141.153,audio_features,0ADG9OgdVTL7fgREP75BrZ,spotify:track:0ADG9OgdVTL7fgREP75BrZ,https://api.spotify.com/v1/tracks/0ADG9OgdVTL7...,https://api.spotify.com/v1/audio-analysis/0ADG...,224031,4
40190,0.611,0.352,10,-9.777,1,0.0236,0.75000,0.000000,0.1130,0.1380,99.971,audio_features,2mOtx6P21hecOcPoVuZbzF,spotify:track:2mOtx6P21hecOcPoVuZbzF,https://api.spotify.com/v1/tracks/2mOtx6P21hec...,https://api.spotify.com/v1/audio-analysis/2mOt...,217840,4
40191,0.806,0.734,7,-7.192,1,0.1200,0.00935,0.000000,0.1730,0.7180,113.106,audio_features,2VxAfqI3vIOaPSlECBNJa8,spotify:track:2VxAfqI3vIOaPSlECBNJa8,https://api.spotify.com/v1/tracks/2VxAfqI3vIOa...,https://api.spotify.com/v1/audio-analysis/2VxA...,156831,4


In [115]:
features.drop(['analysis_url','track_href','type','uri'], axis=1,inplace=True)

In [116]:
features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,40193.000000,4.019300e+04,40193.000000
mean,0.702381,0.652724,5.490658,-6.308446,0.539547,0.132350,0.227793,0.006556,0.162557,0.533651,116.303508,1.962902e+05,3.990745
std,0.135682,0.153307,3.557019,2.258734,0.498440,0.112401,0.224861,0.049807,0.116627,0.217208,26.311783,3.721559e+04,0.276310
min,0.153000,0.031600,0.000000,-23.023000,0.000000,0.023200,0.000055,0.000000,0.018300,0.036000,38.796000,3.013300e+04,1.000000
25%,0.617000,0.557000,2.000000,-7.329000,0.000000,0.047600,0.055800,0.000000,0.092200,0.367000,97.949000,1.733330e+05,4.000000
50%,0.714000,0.665000,6.000000,-6.032000,1.000000,0.086000,0.150000,0.000000,0.118000,0.536000,109.891000,1.954290e+05,4.000000
75%,0.804000,0.768000,8.000000,-4.860000,1.000000,0.191000,0.327000,0.000020,0.188000,0.708000,128.924000,2.163910e+05,4.000000
max,0.974000,0.990000,11.000000,-0.793000,1.000000,0.966000,0.993000,0.918000,0.973000,0.989000,216.821000,1.336000e+06,5.000000


In [118]:
print(features.isna().sum())
print(features.info())

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
id                  0
duration_ms         0
time_signature      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40193 entries, 0 to 40192
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      40193 non-null  float64
 1   energy            40193 non-null  float64
 2   key               40193 non-null  int64  
 3   loudness          40193 non-null  float64
 4   mode              40193 non-null  int64  
 5   speechiness       40193 non-null  float64
 6   acousticness      40193 non-null  float64
 7   instrumentalness  40193 non-null  float64
 8   liveness          40193 non-null  float64
 9   valence           40193 non-null  float64
 10  t

In [119]:
features.drop_duplicates(inplace=True)

In [121]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5044 entries, 0 to 40190
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      5044 non-null   float64
 1   energy            5044 non-null   float64
 2   key               5044 non-null   int64  
 3   loudness          5044 non-null   float64
 4   mode              5044 non-null   int64  
 5   speechiness       5044 non-null   float64
 6   acousticness      5044 non-null   float64
 7   instrumentalness  5044 non-null   float64
 8   liveness          5044 non-null   float64
 9   valence           5044 non-null   float64
 10  tempo             5044 non-null   float64
 11  id                5044 non-null   object 
 12  duration_ms       5044 non-null   int64  
 13  time_signature    5044 non-null   int64  
dtypes: float64(9), int64(4), object(1)
memory usage: 591.1+ KB


In [122]:
df_merge = pd.merge(df, features, left_on="ID", right_on="id", how="inner")
df_merge

,Position,Track Name,Artist,Streams,URL,Date,Start Week,End Week,ID,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,1,Scooter,Yssi SB,2375687,https://open.spotify.com/track/4eWeOwo0TB0vkdr...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,4eWeOwo0TB0vkdrUhy4zbM,0.781,...,1,0.1410,0.262,0.000000,0.4280,0.879,159.981,4eWeOwo0TB0vkdrUhy4zbM,278000,4
1,2,Mood (feat. iann dior),24kGoldn,1614485,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-23--2020-10-30,2020-10-23,2020-10-30,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
2,1,Mood (feat. iann dior),24kGoldn,1637451,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-16--2020-10-23,2020-10-16,2020-10-23,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
3,1,Mood (feat. iann dior),24kGoldn,1693876,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-09--2020-10-16,2020-10-09,2020-10-16,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
4,1,Mood (feat. iann dior),24kGoldn,1787537,https://open.spotify.com/track/3tjFYV6RSFtuktY...,2020-10-02--2020-10-09,2020-10-02,2020-10-09,3tjFYV6RSFtuktYl3ZtYcq,0.700,...,0,0.0369,0.221,0.000000,0.2720,0.756,90.989,3tjFYV6RSFtuktYl3ZtYcq,140526,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40188,174,Man With The Bag,Jessie J,123048,https://open.spotify.com/track/18g50kFt5uwVeCz...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,18g50kFt5uwVeCzQbrCvAO,0.622,...,0,0.0845,0.520,0.000000,0.3750,0.766,140.561,18g50kFt5uwVeCzQbrCvAO,161973,4
40189,176,Christmas Day,Dido,122741,https://open.spotify.com/track/1DGYmHLYkwyCmvD...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,1DGYmHLYkwyCmvDEL3ctti,0.664,...,1,0.0335,0.266,0.000031,0.0762,0.330,80.969,1DGYmHLYkwyCmvDEL3ctti,242893,4
40190,182,Christmas Lights - Live from Spotify London,Coldplay,119168,https://open.spotify.com/track/4QVdnpSO5JGMFOE...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,4QVdnpSO5JGMFOEl6ys4RQ,0.295,...,1,0.0327,0.207,0.000396,0.8580,0.224,132.062,4QVdnpSO5JGMFOEl6ys4RQ,268521,4
40191,195,Christmas (Baby Please Come Home),Darlene Love,114594,https://open.spotify.com/track/46pF1zFimM582ss...,2016-12-23--2016-12-30,2016-12-23,2016-12-30,46pF1zFimM582ss1PrMy68,0.325,...,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,46pF1zFimM582ss1PrMy68,166453,4


In [123]:
df_merge.to_csv("..\\data\\processed\\top200_2017_2020_w_features.csv", index=False)

## 3.
## Data Visualization and Modelling

In [127]:
# Load the data and transform date-time columns
df = pd.read_csv("..\\data\\processed\\top200_2017_2020_w_features.csv")
df[['Start Week', 'End Week']] = df[['Start Week', 'End Week']].apply(pd.to_datetime, format="%Y-%m-%d")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40193 entries, 0 to 40192
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Position          40193 non-null  int64         
 1   Track Name        40193 non-null  object        
 2   Artist            40193 non-null  object        
 3   Streams           40193 non-null  int64         
 4   URL               40193 non-null  object        
 5   Date              40193 non-null  object        
 6   Start Week        40193 non-null  datetime64[ns]
 7   End Week          40193 non-null  datetime64[ns]
 8   ID                40193 non-null  object        
 9   danceability      40193 non-null  float64       
 10  energy            40193 non-null  float64       
 11  key               40193 non-null  int64         
 12  loudness          40193 non-null  float64       
 13  mode              40193 non-null  int64         
 14  speechiness       4019